In [2]:
import pandas as pd

In [5]:
# Load clinical trials data
clinical_trials = pd.read_csv('../data/clinicaltrials_data/ctg-studies.csv')

# Load patient data
patients = pd.read_csv('../data/preprocess_patients.csv')


In [7]:
patients.head(2)

,Id,GENDER,Age
0,e93300bf-3a53-55c0-bd38-2ede59462f21,F,17
1,6b0b0021-df03-cbb6-305a-11ec40da7af4,M,48


In [8]:
clinical_trials.head(2)

,NCT Number,Study Title,Study URL,Study Status,Conditions,Interventions,Sex,Age
0,NCT04687176,Frontline Oral Arsenic Trioxide for APL,https://clinicaltrials.gov/study/NCT04687176,RECRUITING,Acute Promyelocytic Leukemia,DRUG: Oral Arsenic Trioxide Formulation,ALL,"CHILD, ADULT, OLDER_ADULT"
1,NCT06400576,Smartphone & Headphone Effects on Baropodometr...,https://clinicaltrials.gov/study/NCT06400576,RECRUITING,Smartphone Addiction|Postural,Defect|Dual Task,OTHER: Headphone and Smartphone Intervention,ALL


In [9]:
# Function to map patient's age to the trial's age group
def map_age_to_group(age):
    if age <= 17:
        return 'CHILD'
    elif 18 <= age <= 64:
        return 'ADULT'
    else:
        return 'OLDER_ADULT'

# Function to check if the patient's gender matches the trial's gender criteria
def gender_matches(patient_gender, trial_sex):
    if trial_sex == 'ALL':
        return True
    elif trial_sex == 'MALE' and patient_gender == 'M':
        return True
    elif trial_sex == 'FEMALE' and patient_gender == 'F':
        return True
    return False


In [10]:
# List to store the final matching results
results = []

# Loop through each patient
for _, patient in patients.iterrows():
    patient_id = patient['Id']
    patient_age = patient['Age']
    patient_gender = patient['GENDER']
    
    # Map the patient's age to the trial's age group
    patient_age_group = map_age_to_group(patient_age)
    
    eligible_trials = []
    
    # Loop through each clinical trial
    for _, trial in clinical_trials.iterrows():
        trial_id = trial['NCT Number']
        trial_name = trial['Study Title']
        trial_sex = trial['Sex']
        trial_age_group = trial['Age']
        
        # Check if the patient's gender and age group match the trial's criteria
        if gender_matches(patient_gender, trial_sex) and patient_age_group == trial_age_group:
            eligible_trials.append({
                "trialId": trial_id,
                "trialName": trial_name
            })
    
    # Append the patient's results
    results.append({
        "patientId": patient_id,
        "eligibleTrials": eligible_trials
    })

# Convert the results to a DataFrame or export as needed
results_df = pd.DataFrame(results)

# Example: Export to a CSV
results_df.to_csv('../data/matching_results.csv', index=False)
